In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#!pip install pycaret
# from imblearn.over_sampling import RandomOverSampler
from pycaret.classification import *
#from pycaret.regression import *

# Importing the data - only use *.ML.csv files!!!
If you have *.MLReady.csv then [click here](#MLReady)

In [111]:
df=pd.read_csv("../../data/FM_Acq_Perf_FMAC_EPOCH2_ML.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,id,origChannel,origIntRate,origUPB,origDate,firstPmtDate,origLTV,numBorrowers,...,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,1133339,1133339,100209956357,2,5.125,348000,2009-02-01,2009-04-01,87.0,1.0,...,5.04,5.13,0.041667,2.87,-0.125,-0.085,-0.005,-0.023810,-0.016865,-0.000975
1,1133340,1133340,100392775700,3,4.625,195000,2009-02-01,2009-04-01,52.0,2.0,...,5.04,5.13,0.041667,2.87,-0.625,0.415,-0.505,-0.119048,0.082341,-0.098441
2,1133341,1133341,100621428854,2,4.875,342000,2009-02-01,2009-04-01,80.0,1.0,...,5.04,5.13,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708
3,1133342,1133342,100720959726,1,5.375,93000,2009-02-01,2009-05-01,70.0,1.0,...,5.04,5.13,0.041667,2.87,0.125,-0.335,0.245,0.023810,-0.066468,0.047758
4,1133343,1133343,100783704799,1,4.875,182000,2009-02-01,2009-04-01,76.0,2.0,...,5.04,5.13,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708


## Remove columns not involved in 'Foreclosure or Not' query

In [112]:
df.drop(['Unnamed: 0','Unnamed: 0.1','id','firstPmtDate','rptPeriod','currIntRate','currUPB','loanAge','monMatur',\
         'zeroBalDate','lastPdInstDate','forecloDate','dispDate','forecloCost','propRepCost','recovCosts',\
         'miscCost','holdTaxCost','saleProceed','credEnhProceed','repurchProceed','otherForecloProceed',\
         'nonIntUPB','prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator',\
         'deliqGood','deliqBad','deliqMax'],1,inplace=True)

## Engineer originDate into Date features

In [113]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [114]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [115]:
df.drop(['origDate'],1,inplace=True)

In [80]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2009,2
1,2009,2
2,2009,2
3,2009,2
4,2009,2


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [116]:
df = df[~(df['zeroBalCode'].isin(['6','16', '3', '2', '15']))]

In [117]:
df.zeroBalCode.unique()

array([9., 1.])

In [118]:
df['zeroBalCode'] = df.zeroBalCode.replace(1,0)
df['zeroBalCode'] = df.zeroBalCode.replace(9,1)

In [119]:
df.zeroBalCode.unique()

array([1., 0.])

In [120]:
#Checking datatypes of individual feature
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112532 entries, 0 to 119695
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   origChannel         112532 non-null  int64  
 1   origIntRate         112532 non-null  float64
 2   origUPB             112532 non-null  int64  
 3   origLTV             112532 non-null  float64
 4   numBorrowers        112532 non-null  float64
 5   origDebtIncRatio    112532 non-null  float64
 6   borrCreditScore     112532 non-null  float64
 7   loanPurp            112532 non-null  int64  
 8   zipCode             112532 non-null  int64  
 9   pMIperct            112532 non-null  float64
 10  mortInsType         112532 non-null  float64
 11  bestCreditScore     112532 non-null  float64
 12  worstCreditScore    112532 non-null  float64
 13  avgCreditScore      112532 non-null  float64
 14  bankNumber          112532 non-null  int64  
 15  stateNumber         112532 non-nul

# Review values for dates and slice (if needed) the timeframe you want

In [121]:
df.origYear.unique()

array([2009, 2010, 2011, 2012, 2013], dtype=int64)

In [122]:
df['origMonth'].unique()

array([ 2,  3,  5,  4,  6,  7,  8,  9, 10, 11, 12,  1], dtype=int64)

# Coerce the column dtype (int in this case)

In [123]:
df = df.astype({'origLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'borrCreditScore':'int','mortInsType':'int','bestCreditScore':'int',\
               'worstCreditScore':'int','avgCreditScore':'int','zeroBalCode':'int'})
#,'zeroBalCode':'int'

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112532 entries, 0 to 119695
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   origChannel         112532 non-null  int64  
 1   origIntRate         112532 non-null  float64
 2   origUPB             112532 non-null  int64  
 3   origLTV             112532 non-null  int32  
 4   numBorrowers        112532 non-null  int32  
 5   origDebtIncRatio    112532 non-null  int32  
 6   borrCreditScore     112532 non-null  int32  
 7   loanPurp            112532 non-null  int64  
 8   zipCode             112532 non-null  int64  
 9   pMIperct            112532 non-null  float64
 10  mortInsType         112532 non-null  int32  
 11  bestCreditScore     112532 non-null  int32  
 12  worstCreditScore    112532 non-null  int32  
 13  avgCreditScore      112532 non-null  int32  
 14  bankNumber          112532 non-null  int64  
 15  stateNumber         112532 non-nul

In [125]:
years = ['2011']
years2 = ['2012']

In [126]:
df1 = df.loc[df['origYear'].isin(years)]
df2 = df.loc[df['origYear'].isin(years2)]

In [127]:
df1.head()

,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,zipCode,pMIperct,...,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
77166,3,4.750,112000,80,1,36,697,2,609,0.0,...,0.019108,3.3,-0.050,-0.040,-0.005,-0.010417,-0.008493,-0.001052,2011,1
77167,1,4.600,101000,60,1,32,704,1,547,0.0,...,0.019108,3.3,-0.200,0.110,-0.155,-0.041667,0.023355,-0.032597,2011,1
77168,1,5.375,123000,70,1,30,681,1,606,0.0,...,0.019108,3.3,0.575,-0.665,0.620,0.119792,-0.141189,0.130389,2011,1
77169,1,4.375,185000,79,1,31,804,1,194,0.0,...,0.019108,3.3,-0.425,0.335,-0.380,-0.088542,0.071125,-0.079916,2011,1
77170,3,4.375,176000,78,2,30,732,2,670,0.0,...,0.019108,3.3,-0.425,0.335,-0.380,-0.088542,0.071125,-0.079916,2011,1


In [128]:
df2.shape

(18505, 31)

In [129]:
df1.shape

(14111, 31)

<a id='MLReady'></a>

In [130]:
df1 = df1[['origIntRate', 'origUPB', 'origLTV', 'origDebtIncRatio', 'worstCreditScore', 'zeroBalCode', 'mSA']]

In [131]:
df2 = df2[['origIntRate', 'origUPB', 'origLTV', 'origDebtIncRatio', 'worstCreditScore', 'zeroBalCode', 'mSA']]

In [134]:
df1.to_csv('data/2011.csv')

In [135]:
df2.to_csv('data/2012.csv')